<a href="https://colab.research.google.com/github/szhou52/HS-651/blob/main/30_day_readmission_Bert_500_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.1.3 spark-nlp==3.4.2

     |████████████████████████████████| 214.0 MB 7.8 kB/s 
     |████████████████████████████████| 142 kB 17.5 MB/s 
     |████████████████████████████████| 198 kB 47.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463484 sha256=d6cbc18181103717a932ad46fe1078ca2d33a418a8cf7dc6c04c3a60075d2f76
  Stored in directory: /root/.cache/pip/wheels/ad/8e/49/44c110bb8e008d0778c6577d600d46047c6478ecca3f8f1517
Successfully built pyspark


In [2]:
import sparknlp

spark = sparknlp.start(gpu=True)

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  3.4.2
Apache Spark version:  3.1.3


In [3]:
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [4]:
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import StringIndexer

In [5]:
import pandas as pd
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Slice eath document into parts with 500 (or less) tokens each

In [ ]:
processed_train_combined=spark.read.orc("gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_train")
processed_test_combined=spark.read.orc("gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_test")

In [ ]:
pd_train=processed_train_combined.toPandas()

In [ ]:
pd_test=processed_test_combined.toPandas()

In [ ]:
pd_train.head()

,HADM_ID,features,min(label)
0,107872,"[(1, None, None, [-0.03911421820521355, 0.1889...",0.0
1,109103,"[(1, None, None, [0.05161917209625244, -0.1596...",0.0
2,114950,"[(1, None, None, [-0.033079978078603745, 0.091...",0.0
3,130698,"[(1, None, None, [0.02283741906285286, 0.04524...",0.0
4,168628,"[(1, None, None, [0.045720458030700684, 0.0053...",0.0


In [ ]:
pd_train.label=pd_train['min(label)'].astype("int")
pd_test.label=pd_test['min(label)'].astype("int")

In [ ]:
def combined_to_sep(df):
  HADM_ID=[]
  features=[]
  label=[]
  for i in range(len(df)):
    embedding_list=df['features'][i]
    for n in range(len(embedding_list)):
      single_embedding=embedding_list[n]
      HADM_ID.append(df['HADM_ID'][i])
      features.append(single_embedding[3])
      label.append(df['min(label)'][i])
  df_sep= pd.DataFrame(list(zip(HADM_ID,features,label)),
               columns =['HADM_ID','features','label'])
  return(df_sep)

In [ ]:
pd_train_sep=combined_to_sep(pd_train)
pd_test_sep=combined_to_sep(pd_test)

In [ ]:
pd_train_sep.to_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_train_v2.csv')
pd_test_sep.to_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_test_v2.csv')

In [3]:
pd_train_sep=pd.read_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_train_v2.csv')
pd_test_sep=pd.read_csv('gdrive/MyDrive/Colab_notebook/transformed_data/bert_500_test_v2.csv')

In [4]:
# Drop invalid rows where features='[]'
train_invalid_index=[]
for i in range(len(pd_train_sep)):
  if pd_train_sep['features'][i]=='[]':
    train_invalid_index.append(i)
pd_train_sep=pd_train_sep.drop(train_invalid_index).reset_index(drop=True)

test_invalid_index=[]
for i in range(len(pd_test_sep)):
  if pd_test_sep['features'][i]=='[]':
    test_invalid_index.append(i)
pd_test_sep=pd_test_sep.drop(test_invalid_index).reset_index(drop=True)

Modeling

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
len(pd_train_sep)/5

2200.2

In [6]:
X_train=pd_train_sep.features
y_train=pd_train_sep.label
X_train_trans=[]
for doc in X_train:
  embedding=doc[1:-1]
  embedding_list=embedding.split(",")
  str_to_num_list=[]
  for num_str in embedding_list:
    str_to_num_list.append(float(num_str))
  X_train_trans.append(str_to_num_list)

In [7]:
X_test=pd_test_sep.features
y_test=pd_test_sep.label
X_test_trans=[]
for doc in X_test:
  embedding=doc[1:-1]
  embedding_list=embedding.split(",")
  str_to_num_list=[]
  for num_str in embedding_list:
    str_to_num_list.append(float(num_str))
  X_test_trans.append(str_to_num_list)

In [8]:
# Divide cross validation sets manually so that rows with the same admission IDs will be in the same set
subset_1=pd_train_sep.iloc[0:2201]
subset_2=pd_train_sep.iloc[2201:4401]
subset_3=pd_train_sep.iloc[4401:6602]
subset_4=pd_train_sep.iloc[6602:8802]
subset_5=pd_train_sep.iloc[8802:]

In [ ]:
# Logistic Regression
# Write hyper-parameter tuning from scratch so that parameter k can also be tunned
subset_list=[subset_1,subset_2,subset_3,subset_4,subset_5]
solvers_list=['newton-cg', 'lbfgs', 'liblinear']
c_values_list = [1.0, 0.1, 0.01]
k_list=[0.01,0.05,0.1,0.15]
solver_result=[]
c_values_result=[]
k_list_result=[]
auc_score=[]
for i in range(len(subset_list)):
  val_set=subset_list[i]
  training_set=pd.concat(subset_list[:i]+subset_list[i+1:])
  X_train=training_set.features
  y_train=training_set.label
  X_val=val_set.features
  y_val=val_set.label
  X_train_trans=[]
  for doc in X_train:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
      str_to_num_list.append(float(num_str))
    X_train_trans.append(str_to_num_list)
  X_val_trans=[]
  for doc in X_val:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
      str_to_num_list.append(float(num_str))
    X_val_trans.append(str_to_num_list)
  for s in solvers_list:
    for c in c_values_list:
      for k in k_list:
        logistic_clf_bert=LogisticRegression(solver=s,C=c)
        logistic_clf_bert.fit(X_train_trans,y_train)
        y_pred=logistic_clf_bert.predict_proba(X_val_trans)
        ID_PRED=pd.DataFrame()
        ID_PRED['HADM_ID']=val_set['HADM_ID']
        ID_PRED['pred']=y_pred[:,1]
        ID_PRED['actual']=y_val
        re_combined=ID_PRED.groupby('HADM_ID',as_index=False,sort=False).agg({'pred':['mean','max','count'],'actual':'mean'})
        re_combined.columns=['HADM_ID','pred_mean','pred_max','pred_count','actual']
        re_combined['final_prediction']=(re_combined['pred_max']+re_combined['pred_mean']*re_combined['pred_count']/k)/(1+re_combined['pred_count']/k)
        new_auc=roc_auc_score(re_combined['actual'],re_combined['final_prediction'])
        solver_result.append(s)
        c_values_result.append(c)
        k_list_result.append(k)
        auc_score.append(new_auc)
  df_results=pd.DataFrame(list(zip(solver_result, c_values_result,k_list_result,auc_score)),columns=['solver','c','k','auc'])
  result_groupby=df_results.groupby(['solver','c','k'],as_index=False,sort=False).agg({'auc':'mean'})
  result_groupby.columns=['solver','c','k','auc_mean']
  result_groupby=result_groupby.sort_values(by='auc_mean',ascending=False)
result_groupby

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


,solver,c,k,auc_mean
3,newton-cg,0.10,0.05,0.635741
4,newton-cg,0.10,0.10,0.635575
12,lbfgs,0.10,0.05,0.635443
15,lbfgs,0.01,0.05,0.635435
6,newton-cg,0.01,0.05,0.635415
13,lbfgs,0.10,0.10,0.635332
5,newton-cg,0.10,0.15,0.635299
21,liblinear,0.10,0.05,0.635263
24,liblinear,0.01,0.05,0.635232
7,newton-cg,0.01,0.10,0.635221


In [12]:
logistic_clf_bert=LogisticRegression(solver='newton-cg',C=0.1)
logistic_clf_bert.fit(X_train_trans,y_train)
y_pred_logistic_clf_bert=logistic_clf_bert.predict_proba(X_test_trans)

In [13]:
ID_PRED=pd.DataFrame()
ID_PRED['HADM_ID']=pd_test_sep['HADM_ID']
ID_PRED['pred']=y_pred_logistic_clf_bert[:,1]
ID_PRED['actual']=y_test
re_combined=ID_PRED.groupby('HADM_ID',as_index=False,sort=False).agg({'pred':['mean','max','count'],'actual':'mean'})
re_combined.columns=['HADM_ID','pred_mean','pred_max','pred_count','actual']
re_combined['final_prediction']=(re_combined['pred_max']+re_combined['pred_mean']*re_combined['pred_count']/0.05)/(1+re_combined['pred_count']/0.05)
auc_logistic_clf_bert=roc_auc_score(re_combined['actual'],re_combined['final_prediction'])

In [14]:
auc_logistic_clf_bert

0.649272828799493

In [9]:
# Random Forest
# Write hyper-parameter tuning from scratch so that parameter k can also be tunned
subset_list=[subset_1,subset_2,subset_3,subset_4,subset_5]
max_depth=[5, 10, 15]
min_samples_leaf=[20, 50, 100]
criterion=["gini", "entropy"]
n_estimators=[10,50,100]
k_list=[0.01,0.05,0.1]
max_depth_result=[]
min_samples_leaf_result=[]
criterion_result=[]
n_estimators_result=[]
k_list_result=[]
auc_score=[]
for i in range(len(subset_list)):
  val_set=subset_list[i]
  training_set=pd.concat(subset_list[:i]+subset_list[i+1:])
  X_train=training_set.features
  y_train=training_set.label
  X_val=val_set.features
  y_val=val_set.label
  X_train_trans=[]
  for doc in X_train:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
      str_to_num_list.append(float(num_str))
    X_train_trans.append(str_to_num_list)
  X_val_trans=[]
  for doc in X_val:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
      str_to_num_list.append(float(num_str))
    X_val_trans.append(str_to_num_list)
  for depth in max_depth:
    for leaf in min_samples_leaf:
      for criter in criterion:
        for estimator in n_estimators:
          for k in k_list:
            rb_clf_bert=RandomForestClassifier(max_depth=depth,min_samples_leaf=leaf,criterion=criter,n_estimators=estimator)
            rb_clf_bert.fit(X_train_trans,y_train)
            y_pred=rb_clf_bert.predict_proba(X_val_trans)
            ID_PRED=pd.DataFrame()
            ID_PRED['HADM_ID']=val_set['HADM_ID']
            ID_PRED['pred']=y_pred[:,1]
            ID_PRED['actual']=y_val
            re_combined=ID_PRED.groupby('HADM_ID',as_index=False,sort=False).agg({'pred':['mean','max','count'],'actual':'mean'})
            re_combined.columns=['HADM_ID','pred_mean','pred_max','pred_count','actual']
            re_combined['final_prediction']=(re_combined['pred_max']+re_combined['pred_mean']*re_combined['pred_count']/k)/(1+re_combined['pred_count']/k)
            new_auc=roc_auc_score(re_combined['actual'],re_combined['final_prediction'])
            max_depth_result.append(depth)
            min_samples_leaf_result.append(leaf)
            criterion_result.append(criter)
            n_estimators_result.append(estimator)
            k_list_result.append(k)
            auc_score.append(new_auc)
    df_results=pd.DataFrame(list(zip(max_depth_result, min_samples_leaf_result,criterion_result,n_estimators_result,k_list_result,auc_score)),columns=['max_depth','min_samples_leaf','criterion','n_estimator','k','auc'])
    result_groupby=df_results.groupby(['max_depth','min_samples_leaf','criterion','n_estimator','k'],as_index=False,sort=False).agg({'auc':'mean'})
    result_groupby.columns=['max_depth','min_samples_leaf','criterion','n_estimator','k','auc_mean']
    result_groupby=result_groupby.sort_values(by='auc_mean',ascending=False)
result_groupby

,max_depth,min_samples_leaf,criterion,n_estimator,k,auc_mean
125,15,20,entropy,100,0.10,0.637908
87,10,50,entropy,100,0.01,0.634099
141,15,50,entropy,100,0.01,0.633970
152,15,100,gini,100,0.10,0.633764
134,15,50,gini,100,0.10,0.633742
...,...,...,...,...,...,...
118,15,20,entropy,10,0.05,0.587207
108,15,20,gini,10,0.01,0.584073
63,10,20,entropy,10,0.01,0.575711
119,15,20,entropy,10,0.10,0.573388


In [18]:
rb_clf_bert=RandomForestClassifier(max_depth=5,min_samples_leaf=5,criterion='gini',n_estimators=10)
rb_clf_bert.fit(X_train_trans,y_train)
y_pred_rb_clf_bert=rb_clf_bert.predict_proba(X_test_trans)

In [19]:
ID_PRED=pd.DataFrame()
ID_PRED['HADM_ID']=pd_test_sep['HADM_ID']
ID_PRED['pred']=y_pred_rb_clf_bert[:,1]
ID_PRED['actual']=y_test
re_combined=ID_PRED.groupby('HADM_ID',as_index=False,sort=False).agg({'pred':['mean','max','count'],'actual':'mean'})
re_combined.columns=['HADM_ID','pred_mean','pred_max','pred_count','actual']
re_combined['final_prediction']=(re_combined['pred_max']+re_combined['pred_mean']*re_combined['pred_count']/0.01)/(1+re_combined['pred_count']/0.01)
auc_rb_clf_bert=roc_auc_score(re_combined['actual'],re_combined['final_prediction'])

In [20]:
auc_rb_clf_bert

0.6103589349043745

In [23]:
# SVM-linear
subset_list=[subset_1,subset_2,subset_3,subset_4,subset_5]
c_values_list = [100,10,1.0,0.1,0.01]
k_list=[0.01,0.05,0.1,0.15]
c_values_result=[]
k_list_result=[]
auc_score=[]
for i in range(len(subset_list)):
  val_set=subset_list[i]
  training_set=pd.concat(subset_list[:i]+subset_list[i+1:])
  X_train=training_set.features
  y_train=training_set.label
  X_val=val_set.features
  y_val=val_set.label
  X_train_trans=[]
  for doc in X_train:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
      str_to_num_list.append(float(num_str))
    X_train_trans.append(str_to_num_list)
  X_val_trans=[]
  for doc in X_val:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
      str_to_num_list.append(float(num_str))
    X_val_trans.append(str_to_num_list)
  for c in c_values_list:
    for k in k_list:
      lsvm_clf_bert=LinearSVC(C=c)
      lsvm_clf_bert.fit(X_train_trans,y_train)
      y_pred=lsvm_clf_bert.decision_function(X_val_trans)
      ID_PRED=pd.DataFrame()
      ID_PRED['HADM_ID']=val_set['HADM_ID']
      ID_PRED['pred']=y_pred
      ID_PRED['actual']=y_val
      re_combined=ID_PRED.groupby('HADM_ID',as_index=False,sort=False).agg({'pred':['mean','max','count'],'actual':'mean'})
      re_combined.columns=['HADM_ID','pred_mean','pred_max','pred_count','actual']
      re_combined['final_prediction']=(re_combined['pred_max']+re_combined['pred_mean']*re_combined['pred_count']/k)/(1+re_combined['pred_count']/k)
      new_auc=roc_auc_score(re_combined['actual'],re_combined['final_prediction'])
      c_values_result.append(c)
      k_list_result.append(k)
      auc_score.append(new_auc)
  df_results=pd.DataFrame(list(zip(c_values_result,k_list_result,auc_score)),columns=['c','k','auc'])
  result_groupby=df_results.groupby(['c','k'],as_index=False,sort=False).agg({'auc':'mean'})
  result_groupby.columns=['c','k','auc_mean']
  result_groupby=result_groupby.sort_values(by='auc_mean',ascending=False)
result_groupby

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

,c,k,auc_mean
16,0.01,0.01,0.635990
17,0.01,0.05,0.635903
18,0.01,0.10,0.635651
19,0.01,0.15,0.635488
12,0.10,0.01,0.625295
13,0.10,0.05,0.625163
14,0.10,0.10,0.624973
15,0.10,0.15,0.624754
8,1.00,0.01,0.610297
9,1.00,0.05,0.610076


In [26]:
lsvm_clf_bert=LinearSVC(C=0.01)
lsvm_clf_bert.fit(X_train_trans,y_train)
y_pred_lsvm_clf_bert=lsvm_clf_bert.decision_function(X_test_trans)

In [27]:
ID_PRED=pd.DataFrame()
ID_PRED['HADM_ID']=pd_test_sep['HADM_ID']
ID_PRED['pred']=y_pred_lsvm_clf_bert
ID_PRED['actual']=y_test
re_combined=ID_PRED.groupby('HADM_ID',as_index=False,sort=False).agg({'pred':['mean','max','count'],'actual':'mean'})
re_combined.columns=['HADM_ID','pred_mean','pred_max','pred_count','actual']
re_combined['final_prediction']=(re_combined['pred_max']+re_combined['pred_mean']*re_combined['pred_count']/0.01)/(1+re_combined['pred_count']/0.01)
auc_lsvm_clf_bert=roc_auc_score(re_combined['actual'],re_combined['final_prediction'])

In [28]:
auc_lsvm_clf_bert

0.6446652675365743